# Importing and manipulating data with Pandas
An initial analysis of AirBnB listings in Brussels.

In [1]:
# this cell contains our library imports
import pandas as pd
import altair as alt
import geopandas as geopandas

alt.data_transformers.disable_max_rows()
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [2]:
data = pd.read_csv('listings_brussels_20190813.csv')

## Clean up the data a bit and calculate occupancy information

In [3]:
data = data.query('price < 600') # entries over 600 euro are likely typos or hosts that misunderstood the platform
data = data.query('reviews_per_month > 0 & last_review > "2018-07-31"') # we remove any listings with zero reviews for now to focus only on 'active' listings

In [4]:
def calculate_occupancy_rate(number_of_reviews, minimum_nights):
    # first we set some additional assumptions
    review_rate = 0.5
    days_in_month = 31
    average_stay_length_assumption = 3
    max_occupancy = 0.7

    # calculate occupancy
    average_stay_length = minimum_nights.clip(lower = average_stay_length_assumption)
    occupancy_per_month = number_of_reviews / review_rate * average_stay_length / days_in_month
    
    # cap occupancy at max_occupancy
    exceed_max_occupancy = occupancy_per_month > max_occupancy
    
    occupancy_per_month.loc[exceed_max_occupancy] = 0.7

    # fill missing values with a zero (0)
    occupancy_per_month_fillna = occupancy_per_month.fillna(0)

    return occupancy_per_month_fillna

In [5]:
data['occupancy_rate'] = calculate_occupancy_rate(data['reviews_per_month'], data['minimum_nights'])
data['monthly_rental_income'] =  data['price'] * data['occupancy_rate'] * 31

## Ready to roll

In [6]:
data

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,availability_365,occupancy_rate,monthly_rental_income
0,2352,"Triplex-2chmbrs,grande sdb,terrasse= tous confort",2582,Oda,Molenbeek-Saint-Jean,50.85787,4.30809,Entire home/apt,93,2,15,2018-12-31,0.23,229,0.044516,128.34
1,2354,COURT/Long- 1chmb+1salon+sdd+terras,2582,Oda,Molenbeek-Saint-Jean,50.85697,4.30651,Entire home/apt,78,2,2,2018-10-28,0.05,340,0.009677,23.40
2,42521,Ixelles: spacious bedroom+boudoir+ensuite bath...,185699,Claire,Ixelles,50.81929,4.38790,Private room,45,2,130,2019-07-28,1.19,283,0.230323,321.30
3,45145,B&B Welcome to my place - 2,199370,Erick,Saint-Josse-ten-Noode,50.85300,4.36971,Private room,70,2,3,2019-04-08,0.03,190,0.005806,12.60
5,52796,Bright apartment in Ixelles/Louise,244722,Pierre,Ixelles,50.83356,4.36027,Entire home/apt,74,5,96,2019-07-20,0.93,326,0.300000,688.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8830,37472109,La guarida de Cinquantenaire,282690578,Antonio,Bruxelles,50.84451,4.39488,Entire home/apt,40,1,1,2019-08-07,1.00,2,0.193548,240.00
8839,37490590,Appartement 75m² lumineux & cosy - Saint Antoine,26375325,Pierre,Etterbeek,50.83065,4.38724,Entire home/apt,65,2,1,2019-08-11,1.00,56,0.193548,390.00
8849,37522158,appartement sympathique dans le quartier d'Ixe...,190430531,Pierre,Ixelles,50.82105,4.38667,Private room,28,1,1,2019-08-07,1.00,69,0.193548,168.00
8864,37561594,Bright apartment in the historical center.,283734248,Max,Bruxelles,50.84614,4.35407,Entire home/apt,70,2,1,2019-08-10,1.00,122,0.193548,420.00
